In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import norm
from scipy.stats import gstd as geo_std
import datetime
import os
import supertrend_lib as stlib
import sbeta_lib as sbeta
import smacross_lib as smacross
import CAPM_lib as capm
import LR_run_model_lib as LR_run
import LR_model_lib as LR_models
import yfinance as yf
pd.set_option('display.max_rows', 200)
from IPython.display import display, HTML


In [2]:
# symbols_list = eval_df.Stock.values
def get_company_info(symbols_list):
    info_list = []

    for s in symbols_list:
        shandle = yf.Ticker(s)
        info_list.append(shandle.info)

    return info_list
    
# info_list = get_company_info(['aapl','msft'])
# print(info_list)

In [3]:
def add_update(eval_df,values):
    """ Add a a row to dataframe from a list of data points in the same sequence as the columns

    Args:
        eval_df (_type_): DataFrame receiving the new row
        values_list (_type_): A list of values in the same data types and sequence order as the dataframe columns

    Returns:
        _type_: The updated dataframe
    """
    tmp_df = pd.DataFrame([values], columns=eval_df.columns)
    eval_df = pd.concat([eval_df, tmp_df], ignore_index=True)
    return eval_df

In [4]:
def read_stocklist(filename):
    with open(filename) as f:
        stocklist = f.read()
    return stocklist.strip().split(",")


In [5]:
def create_directory(dir_path):
    # Get the user's home directory
    home_directory = os.path.expanduser('~')
    
    # Define the path for the subdirectory
    subdirectory_path = os.path.join(home_directory, dir_path)
          
    # Check if the directory already exists
    if not os.path.exists(subdirectory_path):
        # If it doesn't exist, create the directory
        os.makedirs(subdirectory_path)
        
    return subdirectory_path

In [6]:
def save_list_to_file(sublist,dir_path='.',list_filename='unamed_list.csv'):
    
    subdirectory_path = create_directory(dir_path)
    full_filename = os.path.join(subdirectory_path,list_filename)
    with open( full_filename, 'w') as f:
        f.write(sublist)
        print(f'"{full_filename}" updated!')

### Create an SQLite Database

In [7]:
import sqlite3

# Function to create an SQLite database and table with a composite primary key
def create_database():
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    # Define table schema with composite primary key
    cursor.execute('''CREATE TABLE IF NOT EXISTS stock_data (
                    "Last Run" TEXT,
                    "Stock" TEXT,
                    "Last Price" REAL,
                    "%Std. Dev." TEXT,
                    "$Std. Dev." TEXT,
                    "Supertrend Winner" BOOLEAN,
                    "Supertrend Result" TEXT,
                    "ST Signal_Date" TEXT,
                    "Days@ST" REAL,
                    "LR Best_Model" TEXT,
                    "LR Next_Day Recomm" TEXT,
                    "SMA Crossed_Up" TEXT,
                    "SMA_X_Date" TEXT,
                    "SMA FastXSlow" TEXT,
                    "Beta" REAL,
                    "%Sharpe Ratio" REAL,
                    "CAPM" REAL,
                    "Daily VaR" REAL,
                    PRIMARY KEY ("Last Run", "Stock")
                    )''')
    conn.commit()
    conn.close()


# create_database()

### Insert Dataframe Data into Database 

In [8]:
import sqlite3

def insert_data_into_database(df):
    # Make a copy of the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # Convert date/time columns to the appropriate format
    df_copy['Last Run'] = pd.to_datetime(df_copy['Last Run']).dt.strftime('%Y-%m-%d')
    df_copy['ST Signal_Date'] = pd.to_datetime(df_copy['ST Signal_Date']).dt.strftime('%Y-%m-%d')
    df_copy['SMA_X_Date'] = pd.to_datetime(df_copy['SMA_X_Date']).dt.strftime('%Y-%m-%d')

    # Convert boolean columns to integers
    df_copy['Supertrend Winner'] = df_copy['Supertrend Winner'].astype(int)

    # Connect to the SQLite database
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()

    # Iterate through each row in the DataFrame
    for index, row in df_copy.iterrows():
        # Check if the combination of 'Date' and 'Stock' already exists in the table
        cursor.execute('''SELECT COUNT(*) FROM stock_data WHERE "Last Run" = ? AND "Stock" = ?''',
                       (row['Last Run'], row['Stock']))
        result = cursor.fetchone()[0]

        # If the combination already exists, skip insertion
        if result > 0:
            print(f"Skipping insertion for row {index+1}: Date {row['Last Run']} and Stock {row['Stock']} already exists.")
        else:
            # Print out the row values before insertion
            # print(f"Inserting row {index+1}: {row}")

            # If the combination does not exist, insert the row into the table
            cursor.execute('''INSERT INTO stock_data VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                           (row['Last Run'], row['Stock'], row['Last Price'], str(row['%Std. Dev.']), str(row['$Std. Dev.']),
                            row['Supertrend Winner'], row['Supertrend Result'], row['ST Signal_Date'], row['Days@ST'],
                            row['LR Best_Model'], row['LR Next_Day Recomm'], row['SMA Crossed_Up'], row['SMA_X_Date'],
                            str(row['SMA FastXSlow']), row['Beta'], row['%Sharpe Ratio'], row['CAPM'], row['Daily VaR']))

    # Commit changes and close connection
    conn.commit()
    conn.close()

# Example usage:
# insert_data_into_database(eval_df)

### Write Dataframe as Table to HTML File

In [9]:
from prettytable import PrettyTable

def df_to_html_file(input_df, output_file_name, table_title):
    
    # Create a PrettyTable object
    table = PrettyTable(input_df.columns.tolist())

    # Add rows to the table
    for _, row in input_df.iterrows():
        table.add_row(row.tolist())

    # Generate the HTML table
    table_text = table.get_html_string()

    # Make a comma separated list of stocks in one line
    stock_list = ', '.join(input_df['Stock'].astype(str))
    
    # Manually set the table ID to "sortable-table"
    table_text = table_text.replace("<table>", '<table id="sortable-table">')

    page_header = ''' 
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
    '''

    page_header = f"{page_header}<title>{table_title}</title>"
    # Add table title
    page_header = f"{page_header}\n   <h2>{table_title}</h2>\n"

    # Add CSS styling
    css_style = '''
        <style>
        table {
            border-collapse: collapse;
            width: 100%;
        }
        th, td {
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: #f2f2f2;
        }
        </style>
    '''

    # Include DataTables JavaScript and CSS
    scripts = '''
    
        <script src="https://code.jquery.com/jquery-3.6.4.min.js"></script>
        <script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
        
        <link rel="stylesheet" href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css">

        <script>
            $(document).ready( function () {
                $('#sortable-table').DataTable({"lengthMenu": [ [10, 25, 50, -1], [10, 25, 50, "All"] ], // Customize the options as needed
                    "pageLength": -1 // Change this number to set the initial number of rows displayed per page
                    });
            });
        </script>
    '''

    # Combine CSS styling and table content
    html_page = page_header + css_style +"\n" + scripts + "\n" + "</head>" +"\n"

    # Wrap the table with a div for DataTables
    html_page = f'{html_page}\n<div class="dataTables_wrapper">\n{table_text}\n</div>\n<div>{stock_list}</div>\n</html>'

    # Save the HTML table to the file
    with open(output_file_name, 'w') as f:
        f.write(html_page)


### Write/Append Dataframe to CSV File

In [10]:
def append_to_csv(df, file_path):
    # Append DataFrame to CSV file
    df.to_csv(file_path, mode='a', header=False, index=False)


def write_to_csv(df, csv_file="Historical_Eval_Runs.csv"):
  """
  Appends a DataFrame to a CSV file, handling headers, duplicates, and sorting.

  Args:
      df (pandas.DataFrame): The DataFrame to append.
      csv_file (str): The path to the CSV file.
  """
  exists = os.path.exists(csv_file)

  # Read existing data (if any)
  if exists:
    existing_df = pd.read_csv(csv_file, skiprows=1, parse_dates=True)
    # Set index only if existing_df is not empty
    if not existing_df.empty:
      existing_df.set_index('Last Run', inplace=True)  # Set index after reading
  else:
    existing_df = pd.DataFrame()

  # Extract column names and data types from the DataFrame (unchanged)
  columns = df.columns.tolist()
  dtypes = df.dtypes.to_dict()

  # Remove duplicates based on 'Last Run'
  df.drop_duplicates(subset='Last Run', inplace=True)
  existing_df.drop_duplicates(subset='Last Run', inplace=True)

  # Combine DataFrames
  combined_df = pd.concat([existing_df, df], sort=False)

  # Sort by 'Last Run' (ascending)
  combined_df = combined_df.sort_values(by='Last Run')

  # Write to CSV file
  combined_df.to_csv(csv_file, mode='a' if exists else 'w', index=False, header=not exists)


In [11]:
def update_models(stock_list):
    ret = LR_models.update_list_of_models(stock_list=stock_list)
    models_df = pd.DataFrame({'Stock' : stock_list,
                              'Model' : [f"Model{i}" for i in ret]})
    
    # print(models_df)
    return models_df

In [12]:
# stocks_from_file = read_stocklist(filename='./stocks_list.txt')
# stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
# dfff = update_models(stocks_from_file)
# dfff

In [13]:
# dfff[dfff['Stock'] == 'AAPL']['Model'].values[0]

In [14]:
def run_supertrend(stock,start_date):
    return stlib.supertrend(stock,start_date=start_date)


In [15]:
def init_eval_table():
    columns = ['Last Run'          ,'Stock'             ,'Last Price'     ,'%Std. Dev.','$Std. Dev.','Supertrend Winner' ,'Supertrend Result','ST Signal_Date', 'Days@ST',
               'LR Best_Model'     ,'LR Next_Day Recomm', 'SMA Crossed_Up','SMA_X_Date'        ,'SMA FastXSlow'    , 'Beta'         , 
               '%Sharpe Ratio'                  ,'CAPM'              ,'Daily VaR'  ]

    dtypes = ['datetime64[ns]'       , str                , float            ,str      ,str        , bool              , str               , str            , float,
              str                  , str                , str              , 'datetime64[ns]'    , str               , float          , 
              float                , float                , float        ]

    # Initialize the DataFrame with empty rows
    eval_df = pd.DataFrame(columns=columns)

    # Convert the column data types
    for col, dtype in zip(columns, dtypes):
        eval_df[col] = eval_df[col].astype(dtype)
        
    return eval_df

In [16]:
def first_date_N_years_ago(years):
    # Calculate start and end dates N years back
    end_date = datetime.date.today() - datetime.timedelta(days=1)
    start_date = end_date.replace(year=end_date.year - years, month=1, day=1)
    return start_date.strftime("%Y-%m-%d")


In [17]:
def load_sp500_list():
    stocks_list_csv = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp_df = pd.DataFrame()
    
    try:
        wiki_data=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies') # Open the link and download S&P company details in a table
        data = wiki_data[0] # All data is stored in first cell
        sp_df = data.sort_values(by=['Symbol'], ascending=True) # Sort the dataframe on ticker in alphabetical ascending order
    except:
        print("Cannot open file", stocks_list_csv)

    # remove the dotted symbols, they are redundant 
    no_dot_symbols = [i for i in sp_df['Symbol'] if i.find('.')==-1]
    sp_df = sp_df[sp_df['Symbol'].isin(no_dot_symbols)]
    
    # returns count_row, count_col, df
    return sp_df.shape[0], sp_df.shape[1], sp_df


In [18]:
m,n,sp_df = load_sp500_list()
# sp_df[sp_df['Symbol'].isin([i for i in sp_df['Symbol'] if i.find('.')>0])]

In [19]:
def get_gstd(stock_data):
    # Calculate daily price changes
    price_changes = stock_data["Close"].pct_change()
    price_changes = price_changes.dropna()

    # Calculate the mean and geometric standard deviation of the daily price changes
    gstd = geo_std(price_changes)

    # Return the mean and geometric standard deviation
    return gstd

def get_sigma(stock_data):
    # Calculate daily price changes
    price_changes = stock_data["Close"].pct_change()

    mu, std = norm.fit(price_changes.dropna())
    return mu, std

In [20]:
def limit_trailing_stop_order_percentage(stock,stock_data,multiple_of_std_dev):
    close_price = stock_data.iloc[-1]['Close']
    mean, std_dev = get_sigma(stock_data)
    std_dev_pct = round(std_dev * 100,2)
    std_dev_dlr = round(close_price*std_dev,2)
    
    # use (std_dev_pct x multiple_of_std_dev) as the % to get the limit price
    
    pct_drop = -round(multiple_of_std_dev * std_dev_pct,2) 
    price_drop = round(close_price * pct_drop/100,2)
    min_trailing_stop_price = close_price + price_drop
    
    limit_price = min_trailing_stop_price + std_dev_dlr
    limit_price = round(limit_price,2)
    
    return close_price, pct_drop, price_drop, min_trailing_stop_price, limit_price, std_dev_pct, std_dev_dlr

In [21]:
def recommendation_table(eval_df,stock_list, lookback_years=1, sma_fast=40, sma_slow=200,run_update_models=False):
    stock_list = [str(s).upper() for s in stock_list]
    # print("Stock List:",stock_list)
    start_date = first_date_N_years_ago(lookback_years)
    today = datetime.datetime.today().strftime("%Y-%m-%d")
    if run_update_models :
        print("Updating Models ...",end='')
        stock_best_model_df = update_models(stock_list=stock_list)
        print("Done!")

    print('Performing Analysis and Recommendations ...',end='')

    for s in stock_list:
        winner,buysell,buysell_date,close_price,stock_data, days_at_ST = run_supertrend(s,start_date)
        if(len(stock_data)<2):
            print(f"{s} data not found. Skipping!",end=",")
            continue
        
        print(f"{s}",end=',')
        mean, std_dev = get_sigma(stock_data)
        # std_dev = get_gstd(stock_data)
        std_dev_pct = round(std_dev * 100,2)
        std_dev_dlr = round(close_price*std_dev,2)
        sma_sig, sma_date, fastXslow = smacross.sma_xing(stock_data,sma_fast,sma_slow)
        beta, market_data = sbeta.get_beta(stock_data)
        CAPM, VaR, Sharpe = capm.CAPM_VaR(stock_data=stock_data,market_data=market_data,bond_mat_duration = lookback_years,stock_beta=beta)
        Sharpe = round(Sharpe,2)
        LR_recommend = LR_run.get_recommendation(stock=s,lookback=lookback_years)
        LR_recommend_str = f"{LR_recommend[0]},{LR_recommend[1]},{LR_recommend[2]}"
        
        # columns = ['Last_Run'          ,'Stock'             ,'Last_Price'     ,'%Std. Dev.','$Std. Dev.','Supertrend_Winner' ,'Supertrend_Result','ST_Signal_Date', 'Days@ST',
        #            'LR_Best_Model'     ,'LR_Next_Day_Recomm', 'SMA_Crossed_Up','SMA_X_Date'             ,'SMA_FastXSlow'     , 'Beta'         , 
        #            'SharpeRatio'                  ,'CAPM'              ,'Daily_VaR'  ]
    
        if run_update_models:
            model_str = stock_best_model_df[stock_best_model_df['Stock'] == s].Model.values[0]
        else:
            model_str = 'N/A'
            
        new_row = [today           , s                  , close_price          ,f"+/-{std_dev_pct}%",f"+/-${std_dev_dlr}",    winner            , buysell         ,    buysell_date ,  days_at_ST,  
                   model_str       , LR_recommend_str   , sma_sig              , sma_date           , fastXslow          ,    beta              , 
                   Sharpe          , CAPM            , VaR  ]    
        
        eval_df = add_update(eval_df=eval_df,values = new_row)
        
    print("Done!")
    return eval_df

In [22]:
def recommend_selling_strategy(lookback_years,stock_list,multiple_of_std_dev):
    """
    This function is used to evaluate the selling strategy of the stocks.
    It will return a DataFrame with the following columns:
    - Stock
    - $Last Price
    - Multiple of Std.Dev. Drop
    - %Std. Dev.
    - $Std. Dev.
    - %Trailing Stop Drop
    - $Trailing Stop Drop
    - $Min. Trailing Stop Price
    - $Recomm. Limit Price Order
    """

    # Initialize the DataFrame with empty rows
    start_date = first_date_N_years_ago(lookback_years)

    columns = ['Stock'          ,'$Last Price' , 'Multiple of Std Drop'    ,'%Std. Dev.','$Std. Dev.','%Trailing Stop Drop' , '$Trailing Stop Drop','$Min. Trailing Stop Price','$Recomm. Limit Price Order']
    dtypes = [str               , float        ,float                      ,float        , float     , float                , float                , float              , float]

    # Initialize the DataFrame with empty rows
    sell_orders_recomm = pd.DataFrame(columns=columns)

    # Convert the column data types
    for col, dtype in zip(columns, dtypes):
        sell_orders_recomm[col] = sell_orders_recomm[col].astype(dtype)


    for s in stock_list:
        winner,buysell,buysell_date,close_price,stock_data, days_at_ST = run_supertrend(s,start_date)
        if(len(stock_data)<2):
            print(f"{s.upper()} data not found. Skipping!",end=",")
            continue
        else:
            close_price, pct_drop, price_drop, min_trailing_stop_price, limit_price, std_dev_pct, std_dev_dlr = \
                limit_trailing_stop_order_percentage(stock=s,stock_data=stock_data,multiple_of_std_dev=multiple_of_std_dev)
            # print(f"{s.upper()} Last Price: ${round(close_price,2)}, %Multiple of Std.Dev. Drop %:{multiple_of_std_dev}, % Trailing Drop:{pct_drop}%, \
            #       Limit Price:${limit_price}, Min Price:${round(min_trailing_stop_price,2)}, Std %:{std_dev_pct}, Std $:{std_dev_dlr}")
            
            new_row = [s.upper()  , f"${round(close_price,2)}"    , f"{multiple_of_std_dev}x"     ,f"+/-{std_dev_pct}%",f"+/-${std_dev_dlr}",    f"{pct_drop}%",    f"${price_drop}"   ,  f"${round(min_trailing_stop_price,2)}"  , f"${limit_price}"]    
            sell_orders_recomm = add_update(eval_df=sell_orders_recomm,values = new_row)
            
    return sell_orders_recomm


In [23]:
def eval_all_sp500(lookback_years = 1,sma_fast = 50, sma_slow = 200, regenerate_models = False ):
    n,m,sp_df = load_sp500_list()
    stocks_sp500 = sp_df.Symbol.values
    stocks_sp500 = [s.strip().upper() for s in list(stocks_sp500)]

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_sp500, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow,run_update_models=regenerate_models)
    
    return eval_df

In [24]:
def eval_list_from_file(filename='./stocks_list.txt',lookback_years = 1,sma_fast = 50, sma_slow = 200 ,regenerate_models = False):
    stocks_from_file = read_stocklist(filename=filename)
    stocks_from_file = [s.strip().upper() for s in list(stocks_from_file)]
    today = datetime.datetime.today().strftime("%Y-%m-%d")

    eval_df = init_eval_table()
    eval_df = recommendation_table(eval_df,
                                stock_list=stocks_from_file, 
                                lookback_years=lookback_years, 
                                sma_fast=sma_fast, 
                                sma_slow=sma_slow,run_update_models=regenerate_models)
    
    return eval_df

In [25]:
def screen_for_buys(eval_df, ignore_supertrend_winners=False):
        if not ignore_supertrend_winners:
                buys_df = eval_df[ (eval_df['Supertrend Winner']==True) &  
                        (eval_df['Supertrend Result']=='Buy') & 
                        (eval_df['LR Next_Day Recomm'] == 'Buy,Buy,Buy') &
                        (eval_df['SMA Crossed_Up']=='Buy')].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])
        else:
                buys_df = eval_df[ (eval_df['Supertrend Result']=='Buy') & 
                        (eval_df['LR Next_Day Recomm'] == 'Buy,Buy,Buy') &
                        (eval_df['SMA Crossed_Up']=='Buy')].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])            
        
        return buys_df

In [26]:
def screen_for_sells(eval_df, ignore_supertrend_winners=False):
        if not ignore_supertrend_winners:
                sells_df = eval_df[ (eval_df['Supertrend Winner']==True) &  
                        ((eval_df['Supertrend Result']=='Sell') |
                        (eval_df['LR Next_Day Recomm'] == 'Sell,Sell,Sell') |
                        (eval_df['SMA Crossed_Up']=='Sell')) ].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])
        else:
                sells_df = eval_df[ (eval_df['Supertrend Result']=='Sell') |
                        (eval_df['LR Next_Day Recomm'] == 'Sell,Sell,Sell') |
                        (eval_df['SMA Crossed_Up']=='Sell')].sort_values(by=['Supertrend Winner','Supertrend Result',
                                                                            'ST Signal_Date','SMA Crossed_Up','SMA_X_Date'],
                                                                        ascending=[False,True,False,True,False])            
        
        return sells_df

In [27]:
results_dir = './eval_results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
fname = f'{results_dir}/Eval_Results_{datetime.datetime.today().strftime("%Y_%m_%dT%I%M%S%p")}.csv'

### Evaluate Stocks List

In [28]:
regenerate_models = True
# symbols_file = './stocks_portfolio.txt'
# symbols_file = './stocks_list.txt'
symbols_file = './stocks_list2.txt'
# symbols_file = './stocks_list3.txt'
# symbols_file = './sectors_etfs.txt'
stocks_list = read_stocklist(symbols_file)  

eval_df = eval_list_from_file(filename=symbols_file,lookback_years=2,sma_fast=50,sma_slow=100, regenerate_models=regenerate_models)
# eval_df_from_file = eval_df
# eval_df = eval_all_sp500(lookback_years=2,sma_fast=50,sma_slow=200, regenerate_models=regenerate_models)
## write_to_csv(eval_df)
eval_df.to_csv(fname)
insert_data_into_database(eval_df)

Updating Models ...AAPL,ABNB,ADBE,ADI,ADP,ADSK,AEP,AMAT,AMD,AMGN,AMZN,ANSS,ASML,AVGO,AZN,BA,BIIB,BKNG,BKR,CAT,CCEP,CCL,CDNS,CDW,CEG,CHTR,CMCSA,COIN,COST,CPRT,CRM,CRWD,CSCO,CSGP,CSX,CTAS,CVS,CVX,DASH,DD,DDOG,DE,DIA,DIS,DLTR,DOW,DXCM,EA,EXC,FANG,FAST,FI,FTNT,GD,GEHC,GFS,GILD,GOOG,GOOGL,GS,HD,HON,IBM,IDXX,ILMN,INTC,INTU,ISRG,JBL,JNJ,JNPR,JPM,KDP,KHC,KLAC,KO,LDOS,LIN,LLY,LRCX,LULU,MAR,MCD,MCHP,MDB,MDLZ,MELI,META,MMM,MNST,MRK,MRNA,MRVL,MSFT,MU,NCLH,NFLX,NKE,NOW,NVDA,NXPI,ODFL,ON,ORCL,ORLY,PANW,PAYX,PCAR,PDD,PEP,PFE,PG,PLTR,PYPL,QCOM,QQQ,RCL,REGN,ROP,ROST,SBUX,SIRI,SMCI,SNPS,SPY,SQ,STX,SWKS,TEAM,TMUS,TPR,TRV,TSLA,TSM,TTD,TXN,TXT,UBER,UNH,V,VRSK,VRTX,VZ,WBA,WBD,WDAY,XEL,XOM,ZS,Done!
Performing Analysis and Recommendations ...AAPL,ABNB,ADBE,ADI,ADP,ADSK,AEP,AMAT,AMD,AMGN,AMZN,ANSS,ASML,AVGO,AZN,BA,BIIB,BKNG,BKR,CAT,CCEP,CCL,CDNS,CDW,CEG,CHTR,CMCSA,COIN,COST,CPRT,CRM,CRWD,CSCO,CSGP,CSX,CTAS,CVS,CVX,DASH,DD,DDOG,DE,DIA,DIS,DLTR,DOW,DXCM,EA,EXC,FANG,FAST,FI,FTNT,GD,GEHC,GFS,GILD,GOOG,GOOGL,GS,HD,

### All Results

In [29]:
print(f"{len(eval_df)} Stocks:")
display(HTML(eval_df.sort_values(['ST Signal_Date','SMA_X_Date'], ascending=False).to_html(index=False)))
ret = df_to_html_file(eval_df,"/var/www/html/home/viewable_pages/stock_evaluation.php","Latest Stock Evaluation: All") 

149 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,TEAM,214.65,+/-4.11%,+/-$8.82,False,Buy,2024-04-11,0.0,Model2,"Sell,Sell,Sell",Sell,2024-04-03,"(50, 100)",2.0294,-0.28,3.51,-10.01
2024-04-11,PEP,168.36,+/-1.08%,+/-$1.82,False,Sell,2024-04-11,0.0,Model2,"Buy,Buy,Buy",Buy,2024-03-27,"(50, 100)",0.4455,-1.68,4.62,-2.55
2024-04-11,TRV,220.40,+/-1.34%,+/-$2.96,True,Sell,2024-04-11,0.0,Model2,"Sell,Sell,Sell",Buy,2023-11-24,"(50, 100)",0.4583,3.52,4.61,-3.22
2024-04-11,FAST,69.88,+/-1.59%,+/-$1.11,False,Sell,2024-04-11,0.0,Model2,"Buy,Buy,Buy",Buy,2023-11-21,"(50, 100)",0.8979,0.86,4.30,-3.79
2024-04-11,CCEP,67.89,+/-1.56%,+/-$1.06,True,Sell,2024-04-10,1.0,Model2,"Buy,Buy,Buy",Buy,2024-01-02,"(50, 100)",0.7423,1.66,4.41,-3.74
2024-04-11,KO,59.05,+/-1.02%,+/-$0.6,False,Sell,2024-04-10,1.0,Model2,"Buy,Buy,Buy",Buy,2024-01-02,"(50, 100)",0.4483,-1.71,4.62,-2.41
2024-04-11,GEHC,88.21,+/-1.92%,+/-$1.69,True,Sell,2024-04-10,1.0,Model2,"Buy,Buy,Buy",Buy,2023-12-19,"(50, 100)",0.9868,6.94,34.56,-4.65
2024-04-11,IBM,185.90,+/-1.34%,+/-$2.5,True,Sell,2024-04-10,1.0,Model2,"Sell,Sell,Sell",Buy,2023-07-05,"(50, 100)",0.5725,3.14,4.53,-3.21
2024-04-11,PCAR,118.88,+/-1.52%,+/-$1.81,True,Sell,2024-04-10,1.0,Model1,"Sell,Sell,Sell",Buy,2023-06-16,"(50, 100)",0.7817,7.06,4.38,-3.72
2024-04-11,PFE,26.34,+/-1.55%,+/-$0.41,False,Sell,2024-04-10,1.0,Model2,"Buy,Buy,Buy",Sell,2023-02-24,"(50, 100)",0.4820,-8.69,4.59,-3.54


### Exit Strategy

In [30]:
lookback_years = 2
# my_stock_list = ['amd','jbl','aapl','nvda','msft','googl','amzn','nflx','meta','adbe','cdns','de','avgo','orcl']
my_stock_list = stocks_list
multiple_of_std_dev = 1.5 # Stop Loss as percentage of Std.Dev.
sell_orders_recomm = recommend_selling_strategy(lookback_years,my_stock_list,multiple_of_std_dev)

print("Order Type: Limit Trailing Stop Loss Percent (%) Orders ONLY:  ")
print("=============================================================")
print(f"{len(sell_orders_recomm)} Stocks:")
display(HTML(sell_orders_recomm.to_html(index=False)))
ret = df_to_html_file(sell_orders_recomm,"/var/www/html/home/viewable_pages/exit_strategy.php","Exit Strategies Recommended <br>Order Type: Limit Trailing Stop Loss Percent (%) Orders ONLY") 

Order Type: Limit Trailing Stop Loss Percent (%) Orders ONLY:  
149 Stocks:


Stock,$Last Price,Multiple of Std Drop,%Std. Dev.,$Std. Dev.,%Trailing Stop Drop,$Trailing Stop Drop,$Min. Trailing Stop Price,$Recomm. Limit Price Order
AAPL,$175.04,1.5x,+/-1.78%,+/-$3.12,-2.67%,$-4.67,$170.37,$173.49
ABNB,$165.42,1.5x,+/-3.17%,+/-$5.24,-4.76%,$-7.87,$157.55,$162.79
ADBE,$484.28,1.5x,+/-2.47%,+/-$11.95,-3.7%,$-17.92,$466.36,$478.31
ADI,$198.64,1.5x,+/-1.97%,+/-$3.91,-2.96%,$-5.88,$192.76,$196.67
ADP,$244.42,1.5x,+/-1.5%,+/-$3.66,-2.25%,$-5.5,$238.92,$242.58
ADSK,$241.06,1.5x,+/-2.45%,+/-$5.9,-3.68%,$-8.87,$232.19,$238.09
AEP,$82.91,1.5x,+/-1.39%,+/-$1.15,-2.08%,$-1.72,$81.19,$82.34
AMAT,$212.98,1.5x,+/-2.68%,+/-$5.7,-4.02%,$-8.56,$204.42,$210.12
AMD,$170.5,1.5x,+/-3.43%,+/-$5.84,-5.14%,$-8.76,$161.74,$167.58
AMGN,$270.0,1.5x,+/-1.36%,+/-$3.66,-2.04%,$-5.51,$264.49,$268.15


### Send to Printer

In [31]:
if symbols_file == './stocks_list.txt':
    eval_df.to_csv("./stocks_eval_df_table.csv")
elif symbols_file == './sectors_etfs.txt':
    eval_df.to_csv("./sectors_etfs_eval_df_table.csv")
    
print_it = False

def send_df_to_printer(df):
    pdf_filename = "full_eval_df_table.pdf"

    fig, ax =plt.subplots(figsize=(12,4))
    ax.axis('tight')
    ax.axis('off')
    the_table = ax.table(cellText=df.values,colLabels=df.columns,loc='top')

    pp = PdfPages(pdf_filename)
    pp.savefig(fig, bbox_inches='tight')
    pp.close()

    os.system("lpr "+pdf_filename)
    
if (print_it):
    send_df_to_printer(eval_df)

### Buy, Buy, and more Buys

In [32]:
import time
time.sleep(0.5)
buys_eval_df= screen_for_buys(eval_df=eval_df,ignore_supertrend_winners=False)
buys_eval_df = buys_eval_df.sort_values('Daily VaR',ascending=False).sort_values('%Sharpe Ratio',ascending=True).sort_values(by='SMA_X_Date', ascending=False)
print(f"{len(buys_eval_df)} Stocks:")
sublist = ','.join(buys_eval_df['Stock'].astype(str))
print(sublist)
save_list_to_file(sublist,'picked_stocks','BuyBuyBuy.csv')
display(HTML(buys_eval_df.to_html(index=False)))
ret = df_to_html_file(buys_eval_df,"/var/www/html/home/viewable_pages/buybuybuy.php","Buy, Buy, Buy Rated Stocks") 

12 Stocks:
CAT,MRK,ANSS,ISRG,RCL,JPM,AMAT,NFLX,NVDA,COIN,LLY,MU
"/home/sabawi/picked_stocks/BuyBuyBuy.csv" updated!


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,CAT,371.98,+/-1.89%,+/-$7.04,True,Buy,2024-01-25,77.0,Model2,"Buy,Buy,Buy",Buy,2024-01-18,"(50, 100)",0.8671,4.90,4.32,-4.57
2024-04-11,MRK,126.15,+/-1.21%,+/-$1.53,True,Buy,2024-03-27,15.0,Model2,"Buy,Buy,Buy",Buy,2024-01-11,"(50, 100)",0.2811,6.18,4.73,-2.94
2024-04-11,ANSS,340.10,+/-2.28%,+/-$7.76,True,Buy,2024-03-21,21.0,Model1,"Buy,Buy,Buy",Buy,2024-01-08,"(50, 100)",1.4379,-0.79,3.92,-5.33
2024-04-11,ISRG,392.26,+/-2.29%,+/-$8.97,True,Buy,2023-11-14,149.0,Model2,"Buy,Buy,Buy",Buy,2023-12-26,"(50, 100)",1.3544,1.09,3.98,-5.48
2024-04-11,RCL,133.75,+/-3.29%,+/-$4.4,True,Buy,2024-03-04,38.0,Model2,"Buy,Buy,Buy",Buy,2023-12-18,"(50, 100)",1.6741,3.65,3.76,-7.97
2024-04-11,JPM,195.43,+/-1.55%,+/-$3.02,True,Buy,2023-11-10,153.0,Model2,"Buy,Buy,Buy",Buy,2023-12-15,"(50, 100)",0.8688,1.27,4.32,-3.68
2024-04-11,AMAT,212.98,+/-2.68%,+/-$5.7,True,Buy,2024-01-19,83.0,Model2,"Buy,Buy,Buy",Buy,2023-12-14,"(50, 100)",1.7106,2.57,3.73,-6.44
2024-04-11,NFLX,628.78,+/-3.4%,+/-$21.36,True,Buy,2023-12-13,120.0,Model2,"Buy,Buy,Buy",Buy,2023-12-07,"(50, 100)",1.5670,1.56,3.83,-8.60
2024-04-11,NVDA,906.16,+/-3.52%,+/-$31.88,True,Buy,2023-12-18,115.0,Model2,"Buy,Buy,Buy",Buy,2023-12-05,"(50, 100)",2.2154,6.83,3.38,-8.55
2024-04-11,COIN,263.01,+/-6.18%,+/-$16.24,True,Buy,2024-02-12,59.0,Model2,"Buy,Buy,Buy",Buy,2023-12-01,"(50, 100)",2.8726,2.91,2.92,-15.23


### Buy, Buy, and more Buys (Not necessarily ST winners)

In [33]:
time.sleep(0.5)
buys_eval_df= screen_for_buys(eval_df=eval_df,ignore_supertrend_winners=True)
buys_eval_df = buys_eval_df.sort_values(by='SMA_X_Date', ascending=False).sort_values('%Sharpe Ratio',ascending=True).sort_values('Daily VaR',ascending=False)
print(f"{len(buys_eval_df)} Stocks:")
print(','.join(buys_eval_df['Stock'].astype(str)))
display(HTML(buys_eval_df.to_html(index=False)))
ret = df_to_html_file(buys_eval_df,"/var/www/html/home/viewable_pages/buybuybuy2.php","Buy, Buy, Buy Rated Stocks (Not necessarily ST winners)") 

26 Stocks:
MRK,JPM,TXN,LLY,CAT,DIS,GOOGL,GOOG,CDNS,BKNG,ANSS,SWKS,CSGP,ISRG,QCOM,KLAC,AMAT,MU,FTNT,PYPL,RCL,NVDA,NFLX,MRNA,TTD,COIN


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,MRK,126.15,+/-1.21%,+/-$1.53,True,Buy,2024-03-27,15.0,Model2,"Buy,Buy,Buy",Buy,2024-01-11,"(50, 100)",0.2811,6.18,4.73,-2.94
2024-04-11,JPM,195.43,+/-1.55%,+/-$3.02,True,Buy,2023-11-10,153.0,Model2,"Buy,Buy,Buy",Buy,2023-12-15,"(50, 100)",0.8688,1.27,4.32,-3.68
2024-04-11,TXN,171.20,+/-1.74%,+/-$2.98,False,Buy,2024-03-01,41.0,Model2,"Buy,Buy,Buy",Buy,2024-01-16,"(50, 100)",1.1126,-1.32,4.15,-4.10
2024-04-11,LLY,759.59,+/-1.75%,+/-$13.29,True,Buy,2023-11-08,155.0,Model2,"Buy,Buy,Buy",Buy,2023-05-05,"(50, 100)",0.5206,10.29,4.57,-4.27
2024-04-11,CAT,371.98,+/-1.89%,+/-$7.04,True,Buy,2024-01-25,77.0,Model2,"Buy,Buy,Buy",Buy,2024-01-18,"(50, 100)",0.8671,4.90,4.32,-4.57
2024-04-11,DIS,117.15,+/-2.02%,+/-$2.36,False,Buy,2024-01-22,80.0,Model2,"Buy,Buy,Buy",Buy,2023-11-30,"(50, 100)",1.1094,-2.41,4.15,-4.75
2024-04-11,GOOGL,159.41,+/-2.15%,+/-$3.42,False,Buy,2024-03-14,28.0,Model2,"Buy,Buy,Buy",Buy,2024-01-09,"(50, 100)",1.3540,1.00,3.98,-5.14
2024-04-11,GOOG,160.79,+/-2.15%,+/-$3.46,False,Buy,2024-03-14,28.0,Model2,"Buy,Buy,Buy",Buy,2024-01-09,"(50, 100)",1.3552,1.07,3.98,-5.14
2024-04-11,CDNS,310.10,+/-2.16%,+/-$6.69,False,Buy,2024-03-01,41.0,Model1,"Buy,Buy,Buy",Buy,2023-01-17,"(50, 100)",1.3535,4.70,3.98,-5.21
2024-04-11,BKNG,3637.87,+/-2.16%,+/-$78.5,False,Buy,2024-03-27,15.0,Model1,"Buy,Buy,Buy",Buy,2023-12-26,"(50, 100)",1.1494,3.37,4.13,-5.22


### Save the Buys

In [34]:
eval_df.to_csv("./stocks_buys_eval_df_table.csv")

### Time to Sell!

In [35]:
sells_eval_df= screen_for_sells(eval_df=eval_df,ignore_supertrend_winners=True)
print(f"{len(sells_eval_df)} Stocks:")
print(','.join(sells_eval_df['Stock'].astype(str)))
display(HTML(sells_eval_df.to_html(index=False)))
ret = df_to_html_file(sells_eval_df,"/var/www/html/home/viewable_pages/time_to_sell.php","Time to Sell") 

119 Stocks:
KHC,AVGO,DE,BKR,DXCM,ORCL,CRWD,SMCI,DD,XOM,GD,CPRT,SNPS,QQQ,CDW,FANG,CEG,META,FI,DASH,LDOS,MSFT,TRV,CCEP,GEHC,IBM,PCAR,TPR,MNST,ROST,PDD,TSM,ASML,JNPR,AMD,COST,WDAY,AMGN,BA,TEAM,PAYX,JBL,XEL,STX,KDP,ADI,MMM,MCHP,SQ,MAR,EXC,AZN,DOW,AEP,ODFL,ABNB,TXT,GS,PEP,FAST,KO,UBER,LIN,PFE,VZ,NCLH,REGN,DIA,ORLY,PANW,HON,CSCO,PG,INTC,CVS,CCL,HD,V,PLTR,ON,TMUS,LULU,JNJ,NXPI,LRCX,CRM,INTU,ILMN,IDXX,CSX,DLTR,MRVL,DDOG,ADSK,ZS,EA,SBUX,ADP,NKE,UNH,MELI,VRSK,MDB,NOW,ADBE,SIRI,MDLZ,CMCSA,VRTX,MCD,GFS,ROP,AAPL,GILD,WBA,BIIB,WBD,TSLA,CHTR


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,KHC,36.58,+/-1.36%,+/-$0.5,True,Buy,2024-03-25,17.0,Model2,"Buy,Buy,Buy",Sell,2024-04-03,"(50, 100)",0.2588,-0.83,4.75,-3.24
2024-04-11,AVGO,1382.46,+/-2.23%,+/-$30.84,True,Buy,2024-03-22,20.0,Model2,"Sell,Sell,Sell",Buy,2023-11-22,"(50, 100)",1.3853,5.95,3.96,-5.39
2024-04-11,DE,412.84,+/-1.88%,+/-$7.77,True,Buy,2024-03-13,29.0,Model1,"Sell,Sell,Sell",Buy,2024-01-31,"(50, 100)",0.7821,0.93,4.38,-4.52
2024-04-11,BKR,33.99,+/-2.25%,+/-$0.76,True,Buy,2024-03-11,31.0,Model1,"Sell,Sell,Sell",Sell,2023-11-17,"(50, 100)",0.6810,2.49,4.45,-5.41
2024-04-11,DXCM,139.62,+/-2.94%,+/-$4.11,True,Buy,2024-03-06,36.0,Model2,"Sell,Sell,Sell",Buy,2023-12-21,"(50, 100)",1.3498,1.32,3.98,-7.02
2024-04-11,ORCL,123.24,+/-1.91%,+/-$2.35,True,Buy,2024-03-01,41.0,Model1,"Sell,Sell,Sell",Buy,2024-03-01,"(50, 100)",0.9561,2.97,4.26,-4.62
2024-04-11,CRWD,317.80,+/-3.41%,+/-$10.84,True,Buy,2024-02-29,42.0,Model1,"Sell,Sell,Sell",Buy,2023-03-30,"(50, 100)",1.6923,3.83,3.75,-8.32
2024-04-11,SMCI,937.28,+/-5.08%,+/-$47.58,True,Buy,2024-02-22,49.0,Model2,"Sell,Sell,Sell",Buy,2023-12-21,"(50, 100)",1.7340,12.41,3.72,-12.50
2024-04-11,DD,76.24,+/-1.85%,+/-$1.41,True,Buy,2024-02-07,64.0,Model1,"Sell,Sell,Sell",Sell,2023-11-08,"(50, 100)",1.0267,-0.85,4.21,-4.40
2024-04-11,XOM,121.79,+/-1.84%,+/-$2.24,True,Buy,2024-01-25,77.0,Model2,"Sell,Sell,Sell",Buy,2024-03-18,"(50, 100)",0.5060,5.75,4.58,-4.47


### All Roads Lead to UP & Safe

In [36]:
time.sleep(0.5)
buys_safe = buys_eval_df[(buys_eval_df['Beta']>0.8) & (buys_eval_df['Beta']<2) ].sort_values(by='SMA_X_Date', ascending=False).sort_values('Daily VaR',ascending=False)
print(f"{len(buys_safe)} Stocks:")
print(','.join(buys_safe['Stock'].astype(str)))
display(HTML(buys_safe.to_html(index=False)))
ret = df_to_html_file(buys_safe,"/var/www/html/home/viewable_pages/all_roads_lead_to_up_safe.php","All Roads Lead to UP & Safe") 

21 Stocks:
JPM,TXN,CAT,DIS,GOOGL,GOOG,CDNS,BKNG,ANSS,SWKS,CSGP,ISRG,QCOM,KLAC,AMAT,MU,FTNT,PYPL,RCL,NFLX,MRNA


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,JPM,195.43,+/-1.55%,+/-$3.02,True,Buy,2023-11-10,153.0,Model2,"Buy,Buy,Buy",Buy,2023-12-15,"(50, 100)",0.8688,1.27,4.32,-3.68
2024-04-11,TXN,171.20,+/-1.74%,+/-$2.98,False,Buy,2024-03-01,41.0,Model2,"Buy,Buy,Buy",Buy,2024-01-16,"(50, 100)",1.1126,-1.32,4.15,-4.10
2024-04-11,CAT,371.98,+/-1.89%,+/-$7.04,True,Buy,2024-01-25,77.0,Model2,"Buy,Buy,Buy",Buy,2024-01-18,"(50, 100)",0.8671,4.90,4.32,-4.57
2024-04-11,DIS,117.15,+/-2.02%,+/-$2.36,False,Buy,2024-01-22,80.0,Model2,"Buy,Buy,Buy",Buy,2023-11-30,"(50, 100)",1.1094,-2.41,4.15,-4.75
2024-04-11,GOOGL,159.41,+/-2.15%,+/-$3.42,False,Buy,2024-03-14,28.0,Model2,"Buy,Buy,Buy",Buy,2024-01-09,"(50, 100)",1.3540,1.00,3.98,-5.14
2024-04-11,GOOG,160.79,+/-2.15%,+/-$3.46,False,Buy,2024-03-14,28.0,Model2,"Buy,Buy,Buy",Buy,2024-01-09,"(50, 100)",1.3552,1.07,3.98,-5.14
2024-04-11,CDNS,310.10,+/-2.16%,+/-$6.69,False,Buy,2024-03-01,41.0,Model1,"Buy,Buy,Buy",Buy,2023-01-17,"(50, 100)",1.3535,4.70,3.98,-5.21
2024-04-11,BKNG,3637.87,+/-2.16%,+/-$78.5,False,Buy,2024-03-27,15.0,Model1,"Buy,Buy,Buy",Buy,2023-12-26,"(50, 100)",1.1494,3.37,4.13,-5.22
2024-04-11,ANSS,340.10,+/-2.28%,+/-$7.76,True,Buy,2024-03-21,21.0,Model1,"Buy,Buy,Buy",Buy,2024-01-08,"(50, 100)",1.4379,-0.79,3.92,-5.33
2024-04-11,SWKS,104.33,+/-2.31%,+/-$2.41,False,Buy,2024-03-12,30.0,Model1,"Buy,Buy,Buy",Buy,2024-01-08,"(50, 100)",1.4942,-2.98,3.88,-5.41


### Save the Safe-Buys as Top Picks

In [37]:
buys_safe.to_csv("./top_picks.csv")

### High Sharpe Ratios (Top 15)

In [38]:
hi_sharpe_df = eval_df.sort_values(['%Sharpe Ratio'], ascending=False)
display(HTML(hi_sharpe_df.head(15).to_html(index=False)))
ret = df_to_html_file(hi_sharpe_df,"/var/www/html/home/viewable_pages/high_sharpe_ratios.php","High Sharpe Ratios:<br>The Sharpe ratio compares the return of an investment with its risk. Generally, the higher the more attractive the risk-adjusted return.") 

Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,SMCI,937.28,+/-5.08%,+/-$47.58,True,Buy,2024-02-22,49.0,Model2,"Sell,Sell,Sell",Buy,2023-12-21,"(50, 100)",1.7340,12.41,3.72,-12.50
2024-04-11,CEG,192.88,+/-2.48%,+/-$4.79,True,Buy,2023-12-19,114.0,Model2,"Sell,Sell,Sell",Buy,2023-06-12,"(50, 100)",0.9095,10.96,7.71,-6.10
2024-04-11,LLY,759.59,+/-1.75%,+/-$13.29,True,Buy,2023-11-08,155.0,Model2,"Buy,Buy,Buy",Buy,2023-05-05,"(50, 100)",0.5206,10.29,4.57,-4.27
2024-04-11,PCAR,118.88,+/-1.52%,+/-$1.81,True,Sell,2024-04-10,1.0,Model1,"Sell,Sell,Sell",Buy,2023-06-16,"(50, 100)",0.7817,7.06,4.38,-3.72
2024-04-11,GEHC,88.21,+/-1.92%,+/-$1.69,True,Sell,2024-04-10,1.0,Model2,"Buy,Buy,Buy",Buy,2023-12-19,"(50, 100)",0.9868,6.94,34.56,-4.65
2024-04-11,NVDA,906.16,+/-3.52%,+/-$31.88,True,Buy,2023-12-18,115.0,Model2,"Buy,Buy,Buy",Buy,2023-12-05,"(50, 100)",2.2154,6.83,3.38,-8.55
2024-04-11,MRK,126.15,+/-1.21%,+/-$1.53,True,Buy,2024-03-27,15.0,Model2,"Buy,Buy,Buy",Buy,2024-01-11,"(50, 100)",0.2811,6.18,4.73,-2.94
2024-04-11,AVGO,1382.46,+/-2.23%,+/-$30.84,True,Buy,2024-03-22,20.0,Model2,"Sell,Sell,Sell",Buy,2023-11-22,"(50, 100)",1.3853,5.95,3.96,-5.39
2024-04-11,XOM,121.79,+/-1.84%,+/-$2.24,True,Buy,2024-01-25,77.0,Model2,"Sell,Sell,Sell",Buy,2024-03-18,"(50, 100)",0.5060,5.75,4.58,-4.47
2024-04-11,VRTX,400.23,+/-1.74%,+/-$6.95,False,Sell,2024-02-06,65.0,Model2,"Buy,Buy,Buy",Buy,2023-04-21,"(50, 100)",0.6402,5.74,4.48,-4.20


### UP the Next Day

In [39]:
LR_Next_Day_Recomm_only = eval_df[eval_df['LR Next_Day Recomm']=='Buy,Buy,Buy'].sort_values('Daily VaR',ascending=False)
print(f"{len(LR_Next_Day_Recomm_only)} Stocks:")
print(','.join(LR_Next_Day_Recomm_only['Stock'].astype(str)))
display(HTML(LR_Next_Day_Recomm_only.to_html(index=False)))
ret = df_to_html_file(LR_Next_Day_Recomm_only,"/var/www/html/home/viewable_pages/up_next_day.php","Linear Reg. Predicted Up_Next_Day_only") 

78 Stocks:
KO,JNJ,PEP,PG,MRK,ROP,HON,AMGN,KHC,VZ,EA,UNH,GILD,VRSK,CSX,TMUS,PFE,ADP,JPM,MNST,CCEP,CVS,ORLY,FAST,HD,CMCSA,TXN,MMM,VRTX,SBUX,LLY,JNPR,CAT,GEHC,DIS,WBA,GOOGL,GOOG,ROST,CDNS,BKNG,ANSS,IDXX,SWKS,CSGP,ISRG,BA,CRM,INTU,INTC,QCOM,MCHP,WDAY,TPR,KLAC,ASML,AMAT,MU,LRCX,SIRI,FTNT,PYPL,PANW,RCL,ON,AMD,WBD,NVDA,NFLX,MRVL,TSLA,MRNA,MELI,NCLH,CCL,TTD,PDD,COIN


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,KO,59.05,+/-1.02%,+/-$0.6,False,Sell,2024-04-10,1.0,Model2,"Buy,Buy,Buy",Buy,2024-01-02,"(50, 100)",0.4483,-1.71,4.62,-2.41
2024-04-11,JNJ,148.79,+/-1.04%,+/-$1.54,False,Sell,2024-03-20,22.0,Model2,"Buy,Buy,Buy",Buy,2024-01-25,"(50, 100)",0.3046,-3.69,4.72,-2.41
2024-04-11,PEP,168.36,+/-1.08%,+/-$1.82,False,Sell,2024-04-11,0.0,Model2,"Buy,Buy,Buy",Buy,2024-03-27,"(50, 100)",0.4455,-1.68,4.62,-2.55
2024-04-11,PG,155.84,+/-1.15%,+/-$1.8,False,Sell,2024-04-03,8.0,Model1,"Buy,Buy,Buy",Buy,2024-02-01,"(50, 100)",0.4430,-1.81,4.62,-2.71
2024-04-11,MRK,126.15,+/-1.21%,+/-$1.53,True,Buy,2024-03-27,15.0,Model2,"Buy,Buy,Buy",Buy,2024-01-11,"(50, 100)",0.2811,6.18,4.73,-2.94
2024-04-11,ROP,539.64,+/-1.31%,+/-$7.06,False,Sell,2024-01-31,71.0,Model2,"Buy,Buy,Buy",Buy,2023-05-03,"(50, 100)",0.7901,1.05,4.38,-3.11
2024-04-11,HON,196.87,+/-1.34%,+/-$2.63,False,Sell,2024-04-04,7.0,Model1,"Buy,Buy,Buy",Sell,2024-04-04,"(50, 100)",0.7955,-1.55,4.37,-3.15
2024-04-11,AMGN,270.00,+/-1.36%,+/-$3.66,True,Sell,2024-02-07,64.0,Model2,"Buy,Buy,Buy",Sell,2024-04-10,"(50, 100)",0.3888,1.47,4.66,-3.22
2024-04-11,KHC,36.58,+/-1.36%,+/-$0.5,True,Buy,2024-03-25,17.0,Model2,"Buy,Buy,Buy",Sell,2024-04-03,"(50, 100)",0.2588,-0.83,4.75,-3.24
2024-04-11,VZ,40.16,+/-1.45%,+/-$0.58,False,Sell,2024-04-09,2.0,Model2,"Buy,Buy,Buy",Buy,2023-11-27,"(50, 100)",0.3773,-4.05,4.67,-3.38


### Down the Next Day

In [40]:
LR_Next_Day_Sell_only = eval_df[(eval_df['LR Next_Day Recomm']=='Sell,Buy,Buy') | 
                                (eval_df['LR Next_Day Recomm']=='Sell,Sell,Sell') | 
                                (eval_df['LR Next_Day Recomm']=='Sell,Buy,Sell') | 
                                (eval_df['LR Next_Day Recomm']=='Buy,Sell,Sell')]

print(f"{len(LR_Next_Day_Sell_only)} Stocks:")
print(','.join(LR_Next_Day_Sell_only['Stock'].astype(str)))
display(HTML(LR_Next_Day_Sell_only.to_html(index=False)))
ret = df_to_html_file(LR_Next_Day_Sell_only,"/var/www/html/home/viewable_pages/down_next_day.php","Linear Reg. Predicted Down_Next_Day_only") 

69 Stocks:
AAPL,ABNB,ADBE,ADI,ADSK,AEP,AMZN,AVGO,AZN,BIIB,BKR,CDW,CEG,CHTR,COST,CPRT,CRWD,CSCO,CTAS,CVX,DASH,DD,DDOG,DE,DIA,DLTR,DOW,DXCM,EXC,FANG,FI,GD,GFS,GS,IBM,ILMN,JBL,KDP,LDOS,LULU,MAR,MCD,MDB,MDLZ,META,MSFT,NKE,NOW,NXPI,ODFL,ORCL,PAYX,PCAR,PLTR,QQQ,REGN,SMCI,SNPS,SPY,SQ,STX,TEAM,TRV,TSM,TXT,UBER,XEL,XOM,ZS


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,AAPL,175.04,+/-1.78%,+/-$3.12,False,Sell,2024-01-31,71.0,Model2,"Sell,Sell,Sell",Sell,2024-03-08,"(50, 100)",1.2493,-0.44,4.06,-4.20
2024-04-11,ABNB,165.42,+/-3.17%,+/-$5.24,False,Buy,2024-01-26,76.0,Model1,"Sell,Sell,Sell",Buy,2024-01-09,"(50, 100)",1.7312,0.81,3.72,-7.62
2024-04-11,ADBE,484.28,+/-2.47%,+/-$11.95,False,Sell,2024-02-16,55.0,Model2,"Sell,Sell,Sell",Sell,2024-03-13,"(50, 100)",1.4973,-0.48,3.88,-5.93
2024-04-11,ADI,198.64,+/-1.97%,+/-$3.91,False,Buy,2024-03-12,30.0,Model2,"Sell,Sell,Sell",Buy,2023-12-26,"(50, 100)",1.2800,1.12,4.03,-4.69
2024-04-11,ADSK,241.06,+/-2.45%,+/-$5.9,False,Sell,2024-03-05,37.0,Model2,"Sell,Sell,Sell",Buy,2023-12-13,"(50, 100)",1.6043,-0.59,3.81,-5.79
2024-04-11,AEP,82.91,+/-1.39%,+/-$1.15,False,Buy,2024-02-15,56.0,Model1,"Sell,Sell,Sell",Buy,2023-12-22,"(50, 100)",0.5145,-1.52,4.57,-3.27
2024-04-11,AMZN,189.05,+/-2.58%,+/-$4.87,True,Buy,2024-01-11,91.0,Model2,"Buy,Sell,Sell",Buy,2023-12-05,"(50, 100)",1.6247,1.37,3.79,-6.16
2024-04-11,AVGO,1382.46,+/-2.23%,+/-$30.84,True,Buy,2024-03-22,20.0,Model2,"Sell,Sell,Sell",Buy,2023-11-22,"(50, 100)",1.3853,5.95,3.96,-5.39
2024-04-11,AZN,69.45,+/-1.54%,+/-$1.07,False,Buy,2024-02-20,51.0,Model1,"Sell,Sell,Sell",Sell,2024-04-05,"(50, 100)",0.5080,1.73,4.57,-3.70
2024-04-11,BIIB,203.68,+/-2.38%,+/-$4.84,False,Sell,2024-01-11,91.0,Model2,"Sell,Buy,Buy",Sell,2024-03-05,"(50, 100)",0.8700,-1.00,4.32,-5.20


### Supertrend Winners and Still Supertrending

In [41]:
up_Supertrend = eval_df[(eval_df['Supertrend Result'] == 'Buy') & (eval_df['Supertrend Winner'] == True)].sort_values('ST Signal_Date', ascending=False).sort_values(by='SMA_X_Date', ascending=False)
print(f"{len(up_Supertrend)} Stocks:")
display(HTML(up_Supertrend.to_html(index=False)))
ret = df_to_html_file(up_Supertrend,"/var/www/html/home/viewable_pages/up_supertrend.php","Supertrend Winners and Still Supertrending") 

37 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,KHC,36.58,+/-1.36%,+/-$0.5,True,Buy,2024-03-25,17.0,Model2,"Buy,Buy,Buy",Sell,2024-04-03,"(50, 100)",0.2588,-0.83,4.75,-3.24
2024-04-11,XOM,121.79,+/-1.84%,+/-$2.24,True,Buy,2024-01-25,77.0,Model2,"Sell,Sell,Sell",Buy,2024-03-18,"(50, 100)",0.5060,5.75,4.58,-4.47
2024-04-11,ORCL,123.24,+/-1.91%,+/-$2.35,True,Buy,2024-03-01,41.0,Model1,"Sell,Sell,Sell",Buy,2024-03-01,"(50, 100)",0.9561,2.97,4.26,-4.62
2024-04-11,FANG,208.26,+/-2.39%,+/-$4.97,True,Buy,2023-12-26,107.0,Model2,"Sell,Sell,Sell",Buy,2024-02-23,"(50, 100)",0.7459,4.51,4.41,-5.79
2024-04-11,DE,412.84,+/-1.88%,+/-$7.77,True,Buy,2024-03-13,29.0,Model1,"Sell,Sell,Sell",Buy,2024-01-31,"(50, 100)",0.7821,0.93,4.38,-4.52
2024-04-11,CAT,371.98,+/-1.89%,+/-$7.04,True,Buy,2024-01-25,77.0,Model2,"Buy,Buy,Buy",Buy,2024-01-18,"(50, 100)",0.8671,4.90,4.32,-4.57
2024-04-11,MRK,126.15,+/-1.21%,+/-$1.53,True,Buy,2024-03-27,15.0,Model2,"Buy,Buy,Buy",Buy,2024-01-11,"(50, 100)",0.2811,6.18,4.73,-2.94
2024-04-11,ANSS,340.10,+/-2.28%,+/-$7.76,True,Buy,2024-03-21,21.0,Model1,"Buy,Buy,Buy",Buy,2024-01-08,"(50, 100)",1.4379,-0.79,3.92,-5.33
2024-04-11,ISRG,392.26,+/-2.29%,+/-$8.97,True,Buy,2023-11-14,149.0,Model2,"Buy,Buy,Buy",Buy,2023-12-26,"(50, 100)",1.3544,1.09,3.98,-5.48
2024-04-11,SMCI,937.28,+/-5.08%,+/-$47.58,True,Buy,2024-02-22,49.0,Model2,"Sell,Sell,Sell",Buy,2023-12-21,"(50, 100)",1.7340,12.41,3.72,-12.50


### Supertrending (Winners  or Not)

In [42]:
up_Supertrend = eval_df[eval_df['Supertrend Result'] == 'Buy'].sort_values('ST Signal_Date', ascending=False).sort_values(by='SMA_X_Date', ascending=False)
print(f"{len(up_Supertrend)} Stocks:")
display(HTML(up_Supertrend.to_html(index=False)))
ret = df_to_html_file(up_Supertrend,"/var/www/html/home/viewable_pages/up_supertrend_not_winners.php","Supertrending (Winners  or Not)") 


71 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,MCHP,89.28,+/-2.49%,+/-$2.22,False,Buy,2024-03-07,35.0,Model2,"Buy,Buy,Buy",Sell,2024-04-08,"(50, 100)",1.6127,0.62,3.80,-5.92
2024-04-11,AZN,69.45,+/-1.54%,+/-$1.07,False,Buy,2024-02-20,51.0,Model1,"Sell,Sell,Sell",Sell,2024-04-05,"(50, 100)",0.5080,1.73,4.57,-3.70
2024-04-11,KHC,36.58,+/-1.36%,+/-$0.5,True,Buy,2024-03-25,17.0,Model2,"Buy,Buy,Buy",Sell,2024-04-03,"(50, 100)",0.2588,-0.83,4.75,-3.24
2024-04-11,TEAM,214.65,+/-4.11%,+/-$8.82,False,Buy,2024-04-11,0.0,Model2,"Sell,Sell,Sell",Sell,2024-04-03,"(50, 100)",2.0294,-0.28,3.51,-10.01
2024-04-11,XOM,121.79,+/-1.84%,+/-$2.24,True,Buy,2024-01-25,77.0,Model2,"Sell,Sell,Sell",Buy,2024-03-18,"(50, 100)",0.5060,5.75,4.58,-4.47
2024-04-11,MMM,93.18,+/-1.78%,+/-$1.66,False,Buy,2024-03-12,30.0,Model2,"Buy,Buy,Buy",Sell,2024-03-14,"(50, 100)",0.7960,-4.93,4.37,-4.14
2024-04-11,KDP,30.82,+/-1.23%,+/-$0.38,False,Buy,2024-03-25,17.0,Model2,"Sell,Sell,Sell",Sell,2024-03-08,"(50, 100)",0.4213,-3.66,4.64,-2.86
2024-04-11,CVX,161.89,+/-1.73%,+/-$2.8,False,Buy,2024-01-30,72.0,Model1,"Sell,Buy,Sell",Buy,2024-03-04,"(50, 100)",0.5356,2.69,4.56,-4.15
2024-04-11,TTD,87.70,+/-4.46%,+/-$3.91,False,Buy,2024-03-25,17.0,Model2,"Buy,Buy,Buy",Buy,2024-03-01,"(50, 100)",2.3199,1.83,3.31,-10.52
2024-04-11,XEL,53.49,+/-1.42%,+/-$0.76,False,Buy,2024-03-28,14.0,Model1,"Sell,Sell,Sell",Sell,2024-03-01,"(50, 100)",0.4650,-3.66,4.60,-3.33


### Fast SMA Crossed Slow SMA

In [43]:
Crossed_up = eval_df[eval_df['SMA Crossed_Up'] == 'Buy'].sort_values('SMA_X_Date', ascending=False)
print(f"{len(Crossed_up)} Stocks")
display(HTML(Crossed_up.to_html(index=False)))
ret = df_to_html_file(Crossed_up,"/var/www/html/home/viewable_pages/crossed_up.php","Fast SMA Crossed Slow SMA") 

110 Stocks


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,PEP,168.36,+/-1.08%,+/-$1.82,False,Sell,2024-04-11,0.0,Model2,"Buy,Buy,Buy",Buy,2024-03-27,"(50, 100)",0.4455,-1.68,4.62,-2.55
2024-04-11,XOM,121.79,+/-1.84%,+/-$2.24,True,Buy,2024-01-25,77.0,Model2,"Sell,Sell,Sell",Buy,2024-03-18,"(50, 100)",0.5060,5.75,4.58,-4.47
2024-04-11,CVX,161.89,+/-1.73%,+/-$2.8,False,Buy,2024-01-30,72.0,Model1,"Sell,Buy,Sell",Buy,2024-03-04,"(50, 100)",0.5356,2.69,4.56,-4.15
2024-04-11,TTD,87.70,+/-4.46%,+/-$3.91,False,Buy,2024-03-25,17.0,Model2,"Buy,Buy,Buy",Buy,2024-03-01,"(50, 100)",2.3199,1.83,3.31,-10.52
2024-04-11,ORCL,123.24,+/-1.91%,+/-$2.35,True,Buy,2024-03-01,41.0,Model1,"Sell,Sell,Sell",Buy,2024-03-01,"(50, 100)",0.9561,2.97,4.26,-4.62
2024-04-11,FANG,208.26,+/-2.39%,+/-$4.97,True,Buy,2023-12-26,107.0,Model2,"Sell,Sell,Sell",Buy,2024-02-23,"(50, 100)",0.7459,4.51,4.41,-5.79
2024-04-11,ODFL,219.79,+/-2.31%,+/-$5.07,False,Buy,2024-02-02,69.0,Model1,"Sell,Sell,Sell",Buy,2024-02-14,"(50, 100)",1.2886,1.96,4.03,-5.55
2024-04-11,JBL,138.02,+/-2.34%,+/-$3.23,False,Buy,2024-04-05,6.0,Model1,"Sell,Sell,Sell",Buy,2024-02-12,"(50, 100)",1.2230,5.32,4.07,-5.67
2024-04-11,ADP,244.42,+/-1.5%,+/-$3.66,False,Sell,2024-03-04,38.0,Model1,"Buy,Buy,Buy",Buy,2024-02-07,"(50, 100)",0.8870,-0.56,4.31,-3.56
2024-04-11,PG,155.84,+/-1.15%,+/-$1.8,False,Sell,2024-04-03,8.0,Model1,"Buy,Buy,Buy",Buy,2024-02-01,"(50, 100)",0.4430,-1.81,4.62,-2.71


### SMA Crossed and in Supertrend & Winner

In [52]:
Crossed_up = eval_df[(eval_df['SMA Crossed_Up'] == 'Buy') & (eval_df['Supertrend Result'] == 'Buy') & (eval_df['Supertrend Winner'] == True) ].sort_values('SMA_X_Date', ascending=False)
print(f"{len(Crossed_up)} Stocks:")
# display(Crossed_up.hide_index())
display(HTML(Crossed_up.to_html(index=False)))
ret = df_to_html_file(Crossed_up,"/var/www/html/home/viewable_pages/crossed_up_supertrend.php","SMA Crossed and in Supertrend") 

34 Stocks:


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,XOM,121.79,+/-1.84%,+/-$2.24,True,Buy,2024-01-25,77.0,Model2,"Sell,Sell,Sell",Buy,2024-03-18,"(50, 100)",0.5060,5.75,4.58,-4.47
2024-04-11,ORCL,123.24,+/-1.91%,+/-$2.35,True,Buy,2024-03-01,41.0,Model1,"Sell,Sell,Sell",Buy,2024-03-01,"(50, 100)",0.9561,2.97,4.26,-4.62
2024-04-11,FANG,208.26,+/-2.39%,+/-$4.97,True,Buy,2023-12-26,107.0,Model2,"Sell,Sell,Sell",Buy,2024-02-23,"(50, 100)",0.7459,4.51,4.41,-5.79
2024-04-11,DE,412.84,+/-1.88%,+/-$7.77,True,Buy,2024-03-13,29.0,Model1,"Sell,Sell,Sell",Buy,2024-01-31,"(50, 100)",0.7821,0.93,4.38,-4.52
2024-04-11,CAT,371.98,+/-1.89%,+/-$7.04,True,Buy,2024-01-25,77.0,Model2,"Buy,Buy,Buy",Buy,2024-01-18,"(50, 100)",0.8671,4.90,4.32,-4.57
2024-04-11,MRK,126.15,+/-1.21%,+/-$1.53,True,Buy,2024-03-27,15.0,Model2,"Buy,Buy,Buy",Buy,2024-01-11,"(50, 100)",0.2811,6.18,4.73,-2.94
2024-04-11,ANSS,340.10,+/-2.28%,+/-$7.76,True,Buy,2024-03-21,21.0,Model1,"Buy,Buy,Buy",Buy,2024-01-08,"(50, 100)",1.4379,-0.79,3.92,-5.33
2024-04-11,ISRG,392.26,+/-2.29%,+/-$8.97,True,Buy,2023-11-14,149.0,Model2,"Buy,Buy,Buy",Buy,2023-12-26,"(50, 100)",1.3544,1.09,3.98,-5.48
2024-04-11,DXCM,139.62,+/-2.94%,+/-$4.11,True,Buy,2024-03-06,36.0,Model2,"Sell,Sell,Sell",Buy,2023-12-21,"(50, 100)",1.3498,1.32,3.98,-7.02
2024-04-11,SMCI,937.28,+/-5.08%,+/-$47.58,True,Buy,2024-02-22,49.0,Model2,"Sell,Sell,Sell",Buy,2023-12-21,"(50, 100)",1.7340,12.41,3.72,-12.50


### Test One or a Group of Stocks

In [45]:
stock_to_test = 'swks'
single_stock_df = init_eval_table()
single_stock_list = [stock_to_test]
single_stock_list = ['jbl','aapl','nvda','msft','googl','amzn','nflx','meta','adbe','cdns','de','avgo','orcl']
single_stock_df = recommendation_table(single_stock_df,single_stock_list, lookback_years=5, sma_fast=50, sma_slow=200,run_update_models=False)
display(HTML(single_stock_df.sort_values('SMA_X_Date', ascending=False).to_html(index=False)))

Performing Analysis and Recommendations ...JBL,AAPL,NVDA,MSFT,GOOGL,AMZN,NFLX,META,ADBE,CDNS,DE,AVGO,ORCL,Done!


Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,ADBE,484.28,+/-2.33%,+/-$11.27,False,Sell,2024-02-16,55.0,N/A,"Sell,Sell,Sell",Sell,2024-04-10,"(50, 200)",1.2690,3.03,27.28,-5.64
2024-04-11,ORCL,123.24,+/-1.93%,+/-$2.38,False,Buy,2024-03-01,41.0,N/A,"Sell,Sell,Sell",Buy,2024-03-18,"(50, 200)",0.9353,3.98,21.32,-4.63
2024-04-11,AAPL,175.04,+/-1.98%,+/-$3.47,True,Sell,2024-01-31,71.0,N/A,"Sell,Sell,Sell",Sell,2024-03-13,"(50, 200)",1.2037,6.16,26.12,-4.82
2024-04-11,DE,412.84,+/-2.07%,+/-$8.55,True,Buy,2024-03-13,29.0,N/A,"Sell,Sell,Sell",Sell,2023-10-12,"(50, 200)",0.9833,4.01,22.18,-5.04
2024-04-11,AMZN,189.05,+/-2.19%,+/-$4.14,False,Buy,2024-01-11,91.0,N/A,"Buy,Sell,Sell",Buy,2023-05-26,"(50, 200)",1.0755,3.46,23.83,-5.27
2024-04-11,GOOGL,159.41,+/-1.99%,+/-$3.17,False,Buy,2024-03-14,28.0,N/A,"Buy,Buy,Buy",Buy,2023-05-03,"(50, 200)",1.1381,4.40,24.95,-4.81
2024-04-11,MSFT,427.93,+/-1.89%,+/-$8.07,False,Buy,2023-10-11,183.0,N/A,"Sell,Sell,Sell",Buy,2023-03-21,"(50, 200)",1.1841,5.92,25.77,-4.58
2024-04-11,META,523.16,+/-2.76%,+/-$14.43,True,Buy,2023-11-07,156.0,N/A,"Sell,Sell,Sell",Buy,2023-03-02,"(50, 200)",1.3033,4.52,27.90,-6.76
2024-04-11,NVDA,906.16,+/-3.25%,+/-$29.46,True,Buy,2023-12-18,115.0,N/A,"Buy,Buy,Buy",Buy,2023-01-24,"(50, 200)",1.7443,8.83,35.78,-8.04
2024-04-11,CDNS,310.10,+/-2.22%,+/-$6.89,False,Buy,2024-03-01,41.0,N/A,"Buy,Buy,Buy",Buy,2023-01-10,"(50, 200)",1.2496,7.13,26.94,-5.44


### Filter the Group

In [46]:
single_stock_df= screen_for_buys(eval_df=single_stock_df,ignore_supertrend_winners=True)
display(HTML(single_stock_df.sort_values('SMA_X_Date', ascending=False).to_html(index=False)))

Last Run,Stock,Last Price,%Std. Dev.,$Std. Dev.,Supertrend Winner,Supertrend Result,ST Signal_Date,Days@ST,LR Best_Model,LR Next_Day Recomm,SMA Crossed_Up,SMA_X_Date,SMA FastXSlow,Beta,%Sharpe Ratio,CAPM,Daily VaR
2024-04-11,GOOGL,159.41,+/-1.99%,+/-$3.17,False,Buy,2024-03-14,28.0,N/A,"Buy,Buy,Buy",Buy,2023-05-03,"(50, 200)",1.1381,4.40,24.95,-4.81
2024-04-11,NVDA,906.16,+/-3.25%,+/-$29.46,True,Buy,2023-12-18,115.0,N/A,"Buy,Buy,Buy",Buy,2023-01-24,"(50, 200)",1.7443,8.83,35.78,-8.04
2024-04-11,CDNS,310.10,+/-2.22%,+/-$6.89,False,Buy,2024-03-01,41.0,N/A,"Buy,Buy,Buy",Buy,2023-01-10,"(50, 200)",1.2496,7.13,26.94,-5.44
2024-04-11,NFLX,628.78,+/-2.87%,+/-$18.05,False,Buy,2023-12-13,120.0,N/A,"Buy,Buy,Buy",Buy,2022-11-23,"(50, 200)",1.0480,3.07,23.34,-7.12


In [47]:
import requests
import textwrap
nasdaq_api_key = 'yYCEL8BqzxYUsgG67FTb'

def get_company_info(ticker_symbol):
    ticker_symbol = str(ticker_symbol).upper()
    # Set the endpoint URL and the API key
    url = "https://api.nasdaq.com/api/company/{}/company-profile"
    api_key = nasdaq_api_key

    # Format the endpoint URL with the ticker symbol
    endpoint_url = url.format(ticker_symbol)

    # Set the headers and parameters for the request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299",
        "Accept-Language": "en-US,en;q=0.9"
    }

    params = {
        "apikey": nasdaq_api_key
    }

    # Send the request and get the response
    response = requests.get(endpoint_url, headers=headers, params=params)

    # Parse the response JSON
    response_json = response.json()

    # Get the company full name and sector
    if(response_json["status"]["rCode"] == 200):
        dic = {
        "Ticker" : ticker_symbol,
        "Full_Name" : response_json["data"]["CompanyName"]["value"],
        "Sector" : response_json["data"]["Sector"]["value"],
        "Industry" : response_json["data"]["Industry"]["value"],
        "Region" : response_json["data"]["Region"]["value"],
        "Address" : response_json["data"]["Address"]["value"],
        "Description" : response_json["data"]["CompanyDescription"]["value"]
        }    
        df = pd.DataFrame.from_dict(dic, orient='index').T
        return df
    else:
        print("Return Code : ",response_json["status"]["rCode"])
        print("Error: Symbol Does not exist")
        return pd.DataFrame()
    

company_info = get_company_info('ivr')
if(not company_info.empty):
    print(f"Company  : {company_info.loc[0]['Full_Name']}")
    print(f"Sector   : {company_info.loc[0]['Sector']}")
    print(f"Industry : {company_info.loc[0]['Industry']}")
    print(f"Region   : {company_info.loc[0]['Region']}")

    wrapped_text = textwrap.wrap(company_info.loc[0]['Description'],100)
    for line in wrapped_text:
        print(line)


Company  : INVESCO MORTGAGE CAPITAL INC
Sector   : Real Estate
Industry : Real Estate Investment Trusts
Region   : North America
Invesco Mortgage Capital Inc is an REIT that invests, finances, and manages residential and
commercial mortgage-backed securities and mortgage loans. The company's portfolio consists of
residential mortgage-backed securities for which a U.S. government agency or a federally chartered
corporation guarantees payment of principal and interest, or Agency RMBS; residential mortgage-
backed securities that are not issued or guaranteed by an agency, or nonagency RMBS; commercial
mortgage-backed securities, or CMBS; and residential and commercial mortgage loans. The asset
manager for the company is Invesco Advisers, a subsidiary of Invesco.


In [48]:
import os
import json

def build_sp500_companies_database():
    n, m, sp_df = load_sp500_list()
    stocks_sp500 = sp_df.Symbol.values
    stocks_sp500 = [s.strip().upper() for s in list(stocks_sp500)]
    
    if not os.path.exists("./company_info"):
        os.makedirs("./company_info")
    
    for s in stocks_sp500:
        company_info = get_company_info(s)
        filename = f"./company_info/{s}.json"
        
        with open(filename, "w") as outfile:
            json.dump(company_info, outfile, indent=4)
        
        print(f"Saved {filename}")

    print("Done!")

# build_sp500_companies_database()

In [49]:
    
ret = df_to_html_file(sp_df,"/var/www/html/home/viewable_pages/sp500_companies_info.php","SP500 Companies Information") 
sp_df.tail(50)

KeyError: 'Stock'